In [1]:
from pyexpat import model
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import sys
import cv2
import os
import locale
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import concatenate
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from keras.preprocessing.image import ImageDataGenerator
from sklearn import preprocessing
import locale
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from IPython.display import display
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from deepforest import CascadeForestClassifier
from sklearn.ensemble import RandomForestClassifier

from tensorflow.keras.layers import DepthwiseConv2D
from tensorflow.keras.layers import ReLU, AvgPool2D
# from tensorflow.keras.layers import Input, DepthwiseConv2D
# from tensorflow.keras.layers import Conv2D, BatchNormalization
# from tensorflow.keras.layers import ReLU, AvgPool2D, Flatten, Dense
# from tensorflow.keras import Model

In [2]:
gasdata = pd.read_csv("/Users/Eddie/Downloads/data5 (1).csv")
data_dir = '/Users/Eddie/Downloads/Data5'
imagedata = sorted(os.listdir(data_dir))
print(len(imagedata))
X_data = []
for image in imagedata:
        # print(image)
        img = mpimg.imread('/Users/Eddie/Downloads/Data5/'+image)
        img = img.reshape(320,320,9)
        img = img/255.0
        X_data.append(img)
images = np.array(X_data)
print(images[0])
print(images.shape)

1000
[[[3.8908112e-03 3.9215689e-03 3.8754325e-03 ... 3.8908112e-03
   3.9215689e-03 3.8754325e-03]
  [3.8908112e-03 3.9215689e-03 3.8754325e-03 ... 3.8908112e-03
   3.9215689e-03 3.8754325e-03]
  [3.8908112e-03 3.9215689e-03 3.8754325e-03 ... 3.8908112e-03
   3.9215689e-03 3.8754325e-03]
  ...
  [0.0000000e+00 2.1222609e-03 5.6901196e-04 ... 0.0000000e+00
   2.1991543e-03 9.2272203e-05]
  [0.0000000e+00 2.0299887e-03 7.6893502e-04 ... 0.0000000e+00
   1.9684739e-03 9.3810074e-04]
  [0.0000000e+00 1.9684739e-03 9.3810074e-04 ... 0.0000000e+00
   2.0607461e-03 6.1514805e-04]]

 [[3.8908112e-03 3.9215689e-03 3.8754325e-03 ... 3.8908112e-03
   3.9215689e-03 3.8754325e-03]
  [3.8908112e-03 3.9215689e-03 3.8754325e-03 ... 3.8908112e-03
   3.9215689e-03 3.8754325e-03]
  [3.8908112e-03 3.9215689e-03 3.8754325e-03 ... 3.8908112e-03
   3.9215689e-03 3.8754325e-03]
  ...
  [0.0000000e+00 2.1222609e-03 5.3825456e-04 ... 0.0000000e+00
   2.1683970e-03 2.4605924e-04]
  [0.0000000e+00 2.0915035e-03 

In [3]:
split = train_test_split(gasdata, images, test_size=0.2)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split
trainy = trainAttrX["Gas"]
testy = testAttrX["Gas"]
trainAttrX = trainAttrX.drop(columns=['Gas'])
testAttrX = testAttrX.drop(columns = ['Gas'])
trainAttrX= (trainAttrX - np.min(trainAttrX)) / (np.max(trainAttrX) - np.min(trainAttrX))
testAttrX = (testAttrX - np.min(testAttrX)) / (np.max(testAttrX) - np.min(testAttrX))
print(trainAttrX.shape)
print(testAttrX.shape)
print(trainy.shape)
print(testy.shape)

(800, 2)
(200, 2)
(800,)
(200,)


In [4]:
def create_cnn_alexnet():
    model = Sequential()
    model.add(Conv2D(filters = 96, input_shape = (320, 320, 9),  
                kernel_size = (11, 11), strides = (4, 4),  
                padding = 'valid')) 
    model.add(Activation('relu')) 
    # Max-Pooling  
    model.add(MaxPooling2D(pool_size = (2, 2), 
                strides = (2, 2), padding = 'valid')) 
    # Batch Normalisation 
    model.add(BatchNormalization()) 
    
    # 2nd Convolutional Layer 
    model.add(Conv2D(filters = 256, kernel_size = (11, 11),  
                strides = (1, 1), padding = 'valid')) 
    model.add(Activation('relu')) 
    # Max-Pooling 
    model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2),  
                padding = 'valid')) 
    # Batch Normalisation 
    model.add(BatchNormalization()) 
    
    # 3rd Convolutional Layer 
    model.add(Conv2D(filters = 384, kernel_size = (3, 3),  
                strides = (1, 1), padding = 'valid')) 
    model.add(Activation('relu')) 
    # Batch Normalisation 
    model.add(BatchNormalization()) 
    
    # 4th Convolutional Layer 
    model.add(Conv2D(filters = 384, kernel_size = (3, 3),  
                strides = (1, 1), padding = 'valid')) 
    model.add(Activation('relu')) 
    # Batch Normalisation 
    model.add(BatchNormalization()) 
    
    # 5th Convolutional Layer 
    model.add(Conv2D(filters = 256, kernel_size = (3, 3),  
                strides = (1, 1), padding = 'valid')) 
    model.add(Activation('relu')) 
    # Max-Pooling 
    model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2),  
                padding = 'valid')) 
    # Batch Normalisation 
    model.add(BatchNormalization()) 
    
    # Flattening 
    model.add(Flatten()) 
    
    # 1st Dense Layer 
    model.add(Dense(4096, input_shape = (224*224*3, ))) 
    model.add(Activation('relu')) 
    # Add Dropout to prevent overfitting 
    model.add(Dropout(0.4)) 
    # Batch Normalisation 
    model.add(BatchNormalization()) 
    
    # 2nd Dense Layer 
    model.add(Dense(4096)) 
    model.add(Activation('relu')) 
    # Add Dropout 
    model.add(Dropout(0.4)) 
    # Batch Normalisation 
    model.add(BatchNormalization()) 
    return model

In [5]:
def create_cnn_resnet():
    model = Sequential()
    model.add(Conv2D(64,input_shape = (320,320,9), kernel_size = (7,7), strides = 2, padding = 'valid', activation = 'relu'))
    model.add(AveragePooling2D(pool_size = 2))
    model.add(Conv2D(64, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (3,3), strides  = 1, activation = 'relu', padding  = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, kernel_size = (1,1), strides  = 1, activation = 'relu', padding  = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (3,3), strides  = 1, activation = 'relu', padding  = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, kernel_size = (1,1), strides  = 1, activation = 'relu', padding  = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (3,3), strides  = 1, activation = 'relu', padding  = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, kernel_size = (1,1), strides  = 1, activation = 'relu', padding  = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (3,3), strides  = 1, activation = 'relu', padding  = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, kernel_size = (1,1), strides  = 1, activation = 'relu', padding  = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (3,3), strides  = 1, activation = 'relu', padding  = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, kernel_size = (1,1), strides  = 1, activation = 'relu', padding  = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = (3,3), strides  = 1, activation = 'relu', padding  = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(256, kernel_size = (1,1), strides  = 1, activation = 'relu', padding  = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size = (1,1), strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3,3), strides = 1, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, kernel_size=(1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(128, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3,3), strides = 1, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, kernel_size=(1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(128, kernel_size = (1,1), strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3,3), strides = 1, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, kernel_size=(1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(128, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3,3), strides = 1, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, kernel_size=(1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(128, kernel_size = (1,1), strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3,3), strides = 1, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, kernel_size=(1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(128, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3,3), strides = 1, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, kernel_size=(1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(128, kernel_size = (1,1), strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3,3), strides = 1, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, kernel_size=(1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(128, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(3,3), strides = 1, activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(512, kernel_size=(1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization()) 
    model.add(Conv2D(256,kernel_size=(1,1),strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size=(1,1),strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(1024, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size=(1,1),strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(2048, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size=(1,1),strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(2048, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size=(1,1),strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(2048, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size=(1,1),strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(2048, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size=(1,1),strides = 2, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(2048, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size=(1,1),strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(Conv2D(512,kernel_size = (3,3), strides = 1,activation = 'relu', padding = 'same'))
    model.add(BatchNormalization())
    model.add(Conv2D(2048, kernel_size = (1,1), strides = 1, activation = 'relu', padding = 'valid'))
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size = 2, padding = 'same'))
    model.add(Dropout(0.2))
    model.add(Flatten())

    return model

In [6]:
# MobileNet block
def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

def create_cnn_mobilenet():
    input = Input(shape = (320,320,9))
    x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same')(input)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # main part of the model
    x = mobilnet_block(x, filters = 64, strides = 1)
    x = mobilnet_block(x, filters = 128, strides = 2)
    x = mobilnet_block(x, filters = 128, strides = 1)
    x = mobilnet_block(x, filters = 256, strides = 2)
    x = mobilnet_block(x, filters = 256, strides = 1)
    x = mobilnet_block(x, filters = 512, strides = 2)
    for _ in range (5):
        x = mobilnet_block(x, filters = 512, strides = 1)
    x = mobilnet_block(x, filters = 1024, strides = 2)
    x = mobilnet_block(x, filters = 1024, strides = 1)
    x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)
   # output = Dense (units = 1000, activation = 'softmax')(x)
    output = Flatten()(x)
    model = Model(inputs=input, outputs=output)
    # model.summary()

    return model

In [7]:
alexnet = create_cnn_alexnet()
alexnet.compile(loss = None)
alexnetPredict = alexnet.predict(trainImagesX)
print(alexnetPredict.shape)




25/25 [==============================] - 14s 527ms/step
(800, 4096)


In [8]:
resnet = create_cnn_resnet()
resnetPredict = resnet.predict(trainImagesX)
print(resnetPredict.shape)

25/25 [==============================] - 25s 948ms/step
(800, 2048)


In [9]:
mobilenet = create_cnn_mobilenet()
mobilenetPredict = mobilenet.predict(trainImagesX)
print(mobilenetPredict.shape)

25/25 [==============================] - 13s 483ms/step
(800, 40720)


In [10]:

print(alexnet.summary())
print(resnet.summary())
print(mobilenet.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 78, 78, 96)        104640    
                                                                 
 activation (Activation)     (None, 78, 78, 96)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 39, 39, 96)        0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 39, 39, 96)        384       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 256)       2973952   
                                                                 
 activation_1 (Activation)   (None, 29, 29, 256)       0

In [11]:
alexnetPredict = np.array(alexnetPredict)
resnetPredict = np.array(resnetPredict)
mobilenetPredict = np.array(mobilenetPredict)
trainAttrX = np.array(trainAttrX)
trainAttrX.resize(800,2)
trainData = concatenate([alexnetPredict,resnetPredict,mobilenetPredict,trainAttrX])
trainData = np.array(trainData)
trainData.reshape(-1,1)
print(trainData)

print(trainData.shape)

[[0.0000000e+00 4.8881748e-06 0.0000000e+00 ... 4.6095837e-23
  9.8538011e-01 8.0994153e-01]
 [0.0000000e+00 1.5460462e-05 0.0000000e+00 ... 5.4636954e-23
  8.5233921e-01 6.2573099e-01]
 [6.0781554e-06 0.0000000e+00 0.0000000e+00 ... 2.6073213e-23
  6.7397660e-01 7.3099416e-03]
 ...
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 5.6175063e-23
  6.0964912e-01 4.4005847e-01]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 4.4189953e-23
  9.7807020e-01 8.3040935e-01]
 [0.0000000e+00 2.6063040e-05 0.0000000e+00 ... 7.2351710e-23
  6.5789473e-01 5.0000000e-01]]
(800, 46866)


In [12]:
model = CascadeForestClassifier(n_estimators=50,random_state=1)
model.fit(trainData,trainy)


[2023-12-29 23:07:23.448] Start to fit the model:
[2023-12-29 23:07:23.448] Fitting cascade layer = 0 
[2023-12-29 23:22:57.857] layer = 0  | Val Acc = 98.250 % | Elapsed = 934.410 s
[2023-12-29 23:22:58.042] Fitting cascade layer = 1 
[2023-12-29 23:26:53.042] layer = 1  | Val Acc = 99.000 % | Elapsed = 235.001 s
[2023-12-29 23:26:53.059] Fitting cascade layer = 2 
[2023-12-29 23:30:34.175] layer = 2  | Val Acc = 99.125 % | Elapsed = 221.116 s
[2023-12-29 23:30:34.185] Fitting cascade layer = 3 
[2023-12-29 23:34:14.012] layer = 3  | Val Acc = 99.125 % | Elapsed = 219.827 s
[2023-12-29 23:34:14.013] Early stopping counter: 1 out of 2
[2023-12-29 23:34:14.025] Fitting cascade layer = 4 
[2023-12-29 23:37:59.323] layer = 4  | Val Acc = 99.000 % | Elapsed = 225.297 s
[2023-12-29 23:37:59.323] Early stopping counter: 2 out of 2
[2023-12-29 23:37:59.323] Handling early stopping
[2023-12-29 23:37:59.331] The optimal number of layers: 3


In [13]:
np.set_printoptions(threshold=sys.maxsize)
print(model.get_layer_feature_importances(0))

RuntimeError: Please use the sklearn backend to get the feature importances property for each cascade layer.

In [ ]:
model.save('/Users/Eddie/Downloads/deepforest')

In [ ]:
gasdata = pd.read_csv("/Users/eddie/Downloads/Gas Sensors Data - Sheet1 (3) (1).csv")
result = gasdata['Gas']
gasdata = gasdata.drop(columns=['Gas'])
gasdata= (gasdata - np.min(gasdata)) / (np.max(gasdata) - np.min(gasdata))
images =  sorted(os.listdir("/Users/eddie/Downloads/No Gas 4/No Gas"))
X_dat = []
for image in images:
       #print(image)
       img = mpimg.imread('/Users/eddie/Downloads/No Gas 4/No Gas/'+image)
       img.resize(320,320,9)
       img = img/255.0
       X_dat.append(img)
test_images = np.array(X_dat)
print("gay")
print(test_images[1])
print(test_images.shape)

gay
[[[0.0000000e+00 3.1372549e-03 3.2602844e-03 ... 3.2295270e-03
   3.9215689e-03 0.0000000e+00]
  [3.4755864e-03 3.1987696e-03 3.9215689e-03 ... 3.9215689e-03
   0.0000000e+00 3.2756634e-03]
  [3.2449057e-03 3.9215689e-03 0.0000000e+00 ... 0.0000000e+00
   3.0911188e-03 3.2756634e-03]
  ...
  [3.5986160e-03 3.1064975e-03 3.9215689e-03 ... 3.9215689e-03
   3.0757405e-05 3.5678586e-03]
  [3.1372549e-03 3.9215689e-03 1.5378702e-05 ... 1.5378702e-05
   3.5371012e-03 3.1680122e-03]
  [3.9215689e-03 4.6136101e-05 3.5524799e-03 ... 3.5832373e-03
   3.0603614e-03 3.9215689e-03]]

 [[1.0765091e-04 3.5986160e-03 3.0142253e-03 ... 2.9527105e-03
   3.9215689e-03 1.6916571e-04]
  [3.6293734e-03 2.9065744e-03 3.9215689e-03 ... 3.9215689e-03
   1.2302962e-04 3.6447521e-03]
  [2.9680892e-03 3.9215689e-03 9.2272203e-05 ... 7.6893506e-05
   3.6601308e-03 3.0296040e-03]
  ...
  [3.0296040e-03 3.2910421e-03 3.9215689e-03 ... 3.9215689e-03
   0.0000000e+00 3.1218762e-03]
  [3.2602844e-03 3.9215689e-03 0

In [ ]:
# savemodel
# testAttrX
# testImagesX

alexnetPredictTest = np.array(alexnet.predict(test_images))
resnetPredictTest = np.array(resnet.predict(test_images))
mobilenetPredictTest = np.array(mobilenet.predict(test_images))
print(alexnetPredictTest.shape)
print(resnetPredictTest.shape)
print(mobilenetPredictTest.shape)
print(gasdata.shape)



6/6 [==============================] - 3s 432ms/step
(176, 4096)
(176, 2048)
(176, 40720)
(176, 2)


In [ ]:
alexnetPredictTest = np.array(alexnetPredictTest)
resnetPredictTest = np.array(resnetPredictTest)
mobilenetPredictTest = np.array(mobilenetPredictTest)
gasdata = np.array(gasdata)
trainAttrX.resize(176,2)
testData = concatenate([alexnetPredictTest,resnetPredictTest,mobilenetPredictTest,gasdata])
testData = np.array(testData)
testData.reshape(-1,1)
print(testData)

print(testData.shape)

[[0.0000000e+00 0.0000000e+00 2.3009535e-04 ... 1.8220146e-22
  2.0249221e-02 2.6479751e-02]
 [0.0000000e+00 0.0000000e+00 2.2060507e-04 ... 1.7716298e-22
  2.1806853e-02 2.6479751e-02]
 [0.0000000e+00 0.0000000e+00 2.2190153e-04 ... 1.7929146e-22
  2.0249221e-02 2.6479751e-02]
 ...
 [0.0000000e+00 3.9214090e-05 1.9796153e-04 ... 2.0455493e-22
  1.0000000e+00 1.0000000e+00]
 [0.0000000e+00 1.5338414e-05 1.8607419e-04 ... 2.1479534e-22
  1.0000000e+00 1.0000000e+00]
 [0.0000000e+00 4.8333448e-05 2.4477928e-04 ... 2.1644574e-22
  1.0000000e+00 1.0000000e+00]]
(176, 46866)


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
performance = model.predict(testData)
print(performance)
actual = []
for value in performance: 
    #print(value)
    if(value>=0.5):
            actual.append(1)
    else:
            actual.append(0)
acutal = np.array(actual)
print(result)
accuracy = accuracy_score(result, actual)
print('Accuracy: %f' % accuracy)
precision = precision_score(result, actual)
print('Precision: %f' % precision)
recall = recall_score(result, actual)
print('Recall: %f' % recall)
f1 = f1_score(result, actual)
print('F1 score: %f' % f1)

[2023-12-27 22:43:47.659] Start to evalute the model:
[2023-12-27 22:43:47.995] Evaluating cascade layer = 0 
[2023-12-27 22:43:48.445] Evaluating cascade layer = 1 
[2023-12-27 22:43:48.753] Evaluating cascade layer = 2 
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
0      0
1      0
2      0
3      0
4      0
      ..
171    1
172    1
173    1
174    1
175    1
Name: Gas, Length: 176, dtype: int64
Accuracy: 0.630682
Precision: 0.000000
Recall: 0.000000
F1 score: 0.000000


c:\Users\eddie\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
